# Part 2 - Script writing
This python notebook:
- uses the get_weather_data function to make the API call to OpenWeather;
- uses psycopg2 to connect to a PostgreSQL database (in this case it was hosted on my localhost);
- uses a for loop which iterates over the values of the 'cities' list, and for each city, it appends the relevant values to a list of dictionaries, which is converted to a dataframe item
- uses another for loop to iterate over the dataframe's rows

In [105]:
import requests
import json
import psycopg2
from datetime import datetime, timezone
import pandas as pd

In [100]:
key = ''#redacted for github
base_url = "http://api.openweathermap.org/data/2.5/weather?q={}&appid={}"
conn = psycopg2.connect("dbname=weatherdb user=dgiord password=pass1 host=localhost port=5432")
conn.autocommit=True
cur = conn.cursor()

In [101]:
def get_weather_data(city):
    try:
        response = requests.get(base_url.format(city,key)) #http get request to OpenWeather API
        if response.status_code == 200:                    #in case of positive http response (200), returns the json output
            return response.json()
        else:
            print ("Failed to fetch data for city {city}. Status code: {responde.status_code}")
            return None
    except Exception as e:
        print("Error occurred:{e}")
        return None

query_city: inserts the city id and name in the city table. In case the city_id is already present in the table, the value is ignored. This query can be skipped as long as the only 3 cities we consider are Milano, Bologna and Cagliari, however it's present in case of future data needs.\
\
query_weather: inserts values in the weather table. Although the weather_id column is a serial and should auto-increase, I encountered problems when having some values in the table alread. Therefore, I included a subquery that either adds 1 to the maximum value of the column, or, if it only finds NULL values, adds 0 as weather_id.

In [104]:
query_city = """INSERT INTO public.city
(city_id, city_name)
VALUES (%s, %s)
ON CONFLICT (city_id) DO NOTHING;
"""
query_weather = """INSERT INTO public.weather
    (weather_id, city_id, "timestamp", temperature, min_temperature, max_temperature, humidity, weather_condition, wind_speed)
    VALUES (
    (SELECT COALESCE(MAX(weather_id), 0) + 1 from public.weather), %s, %s, %s, %s, %s, %s, %s, %s);"""

cities = ['Milano', 'Bologna', 'Cagliari'] #can be expanded in case of future data needs

In [103]:
weather_list = []
for city in cities:
    weather_data = get_weather_data(city)
    
    weather_list.append({
        'city_id': weather_data['id'],
        'city_name': weather_data['name'],
        'timestamp': datetime.fromtimestamp(weather_data['dt'], tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'),
        'temperature': weather_data['main']['temp'],
        'min_temperature': weather_data['main']['temp_min'],
        'max_temperature': weather_data['main']['temp_max'],
        'humidity': weather_data['main']['humidity'],
        'weather_condition': weather_data['weather'][0]['main'],
        'wind_speed': weather_data['wind']['speed']
    })

weather_df = pd.DataFrame(weather_list)

In [120]:
for index,row in weather_df.iterrows():
    values_city = (row['city_id'], row['city_name'])
    values_weather = (row['city_id'], row['timestamp'], row['temperature'], row['min_temperature'], 
                      row['max_temperature'], row['humidity'], row['weather_condition'], row['wind_speed'])

    cur.execute(query_city, values_city)
    cur.execute(query_weather, values_weather)

### The script can be considered complete here. The section below is used to answer the questions from Part 1 using Python instead of SQL queries.

As the weather_df dataframe created in the script is reset each time the script is run (therefore it will always only contain 3 rows), I create a new dataframe with a SQL query on my local database. The output is simply a join of all columns in both tables, which gives us a dataframe with all the info we have in the db.

In [218]:
from sqlalchemy import create_engine
#I'm using SQLAlchemy since it's the only connectable supported by pandas
engine = create_engine(f"postgresql://dgiord:{'pass1'}@{'localhost'}:{5432}/{'weatherdb'}") 
query = """
SELECT 
    w.weather_id, 
    c.city_id,
    c.city_name,
    w.timestamp, 
    w.temperature, 
    w.min_temperature, 
    w.max_temperature, 
    w.humidity, 
    w.weather_condition, 
    w.wind_speed 
FROM public.weather w
JOIN public.city c 
ON w.city_id = c.city_id;
"""

df = pd.read_sql(query,engine)
df.head()

,weather_id,city_id,city_name,timestamp,temperature,min_temperature,max_temperature,humidity,weather_condition,wind_speed
0,1,3173435,Milan,2024-10-21 08:18:40+00:00,293.60,292.14,294.85,75,Clouds,0.45
1,2,3181927,Bologna,2024-10-21 08:26:41+00:00,291.00,290.40,293.92,87,Clouds,1.03
2,3,2525471,Provincia di Cagliari,2024-10-21 08:25:13+00:00,295.86,295.30,296.20,66,Clouds,3.60
3,4,3173435,Milan,2024-10-21 08:29:28+00:00,293.75,292.26,294.85,75,Clouds,0.45
4,5,3181927,Bologna,2024-10-21 08:31:34+00:00,291.15,290.95,293.92,87,Clouds,1.03


##### Distinct weather conditions in a given period

In [229]:
#the period can be set by changing these variables. I'm creating a newdf filtered dataframe as all questions refer to a given period, and it will be used for all questions
start_date = '2024-10-01'
end_date = '2024-11-01'
newdf = df[(df['timestamp'] >= start_date) & (df['timestamp']<= end_date)]

# nunique displays the number of unique entries for a specific column, in this case weather_condition
unique = newdf['weather_condition'].nunique()
print(f"Unique conditions in the period {start_date} - {end_date}: {unique}")

Unique conditions in the period 2024-10-01 - 2024-11-01: 2


##### Most common conditions in a given period by city

In [171]:
# group by city and weather condition and count the occurrance for each combination of city and condition
weather_ranking = newdf.groupby(['city_name','weather_condition'])
weather_ranking = weather_ranking.size().reset_index(name='count')

print(weather_ranking)

city_name              weather_condition
Bologna                Clouds               4
Milan                  Clear                2
                       Clouds               2
Provincia di Cagliari  Clouds               4
dtype: int64


##### Temperature averages in a given period per city

In [173]:
# group by city and temperature and find the average
avg_temp = newdf.groupby('city_name')['temperature'].mean().reset_index(name='avg_temp')

print (avg_temp)

               city_name  avg_temp
0                Bologna  291.4025
1                  Milan  294.2575
2  Provincia di Cagliari  295.9900


##### City with the highest temperature in a given period

In [183]:
# nlargest displays the entire rows for the top N values (in this case, 1) of a specific column (in this case, temperature)
max_temp = newdf.nlargest(1,'temperature')

max_temp


,weather_id,city_id,city_name,timestamp,temperature,min_temperature,max_temperature,humidity,weather_condition,wind_speed
8,9,2525471,Provincia di Cagliari,2024-10-21 10:06:13+00:00,296.12,295.34,296.43,66,Clouds,4.12


##### City with the highest daily temperature variation in a given period

In [228]:
# we create a new column called temp_diff, which is the difference between the minimum and maximum temperature, and display the highest value using nlargest

newdf['temp_diff'] = newdf['max_temperature'] - newdf['min_temperature']
max_temp_diff = newdf.nlargest(1,'temp_diff')
max_temp_diff

,weather_id,city_id,city_name,timestamp,temperature,min_temperature,max_temperature,humidity,weather_condition,wind_speed,temp_diff
1,2,3181927,Bologna,2024-10-21,291.0,290.4,293.92,87,Clouds,1.03,3.52


##### City with the strongest wind in a given period

In [184]:
# nlargest displays the entire rows for the top N values (in this case, 1) of a specific column (in this case, wind speed)
max_wind = newdf.nlargest(1,'wind_speed')

max_wind

,weather_id,city_id,city_name,timestamp,temperature,min_temperature,max_temperature,humidity,weather_condition,wind_speed
8,9,2525471,Provincia di Cagliari,2024-10-21 10:06:13+00:00,296.12,295.34,296.43,66,Clouds,4.12
